# Import libraries

In [ ]:
from tensorflow import keras
from keras.models import Sequential 
from keras.layers import Dense, LSTM
from keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os, sys

In [ ]:
# Helper functions

def flatten(l):
    try:
        return flatten(l[0]) + (flatten(l[1:]) if len(l) > 1 else []) if type(l) is list else [l]
    except IndexError:
        return []

In [ ]:
# Declare variables / parameters

FILEPATH = './HDeviceCGM.csv'
FIXED_WINDOW_SIZE = 10
TRAIN_PCT = 0.8

# Understanding the data
In this dataset we will be provided with dataset containing the following columns of importance
1. RecID - unique record ID
2. DeviceDtTmDaysFromEroll (Date time index)
3. GlucoseValue (mg/dL) - the output glucose number
Prepare the dataset, filter out the necessary data and split it to its respective test and training sets.

Within each `DeviceDtTmDaysFromEnroll` group, we need to sort the `DeviceTm` in an ascending order. This is because the input data into the LSTM model requires each subarray to be a windowed data, which should be a time series.

In [ ]:
df = pd.read_csv(FILEPATH) \
     .sort_values(['DeviceDtTmDaysFromEnroll'], ascending=True) \
     .groupby(['DeviceDtTmDaysFromEnroll'], sort=False) \
     .apply(lambda x: x.sort_values(['DeviceTm'], ascending=True)) \
     .reset_index(drop=True)

df

### Checkpoint 1: Study the training data-set and answer the following 
1. How many sets of continuous training data sets do you have available
2. The count of sample points per training data sets. Segment the data-sets into training and testing.

In [ ]:
# filter out unnecessary data

df = df[['RecID', 'DeviceDtTmDaysFromEnroll', 'GlucoseValue']]
df

In [ ]:
df_grouped = df.groupby(by='DeviceDtTmDaysFromEnroll').agg(count=('RecID', pd.Series.nunique)).reset_index()
NO_OF_INSTANCES = len(df_grouped)

print("There are {} continuous training data sets.".format(NO_OF_INSTANCES))
print("The count of sample points per training dataset is shown below.")

df_grouped

In [ ]:
# create a dataset, which is a list of list
dataset = [[] for i in range(NO_OF_INSTANCES)]

for index, row in df.iterrows():
    group_no = row['DeviceDtTmDaysFromEnroll']
    dataset[group_no].append(row['GlucoseValue'])

print("There are {} datasets.".format(len(dataset))) # a list of 11 instances
print("There are {} records in day 0 dataset.".format(len(dataset[0]))) # each containing a list of glucose values

In [ ]:
# split into respective training and test sets
train_size = int(len(dataset) * TRAIN_PCT)
train, test = dataset[0:train_size], dataset[train_size:]

print("There are {} train datasets.".format(len(train))) 
print("There are {} test datasets.".format(len(test))) 

# Data Pre-Processing
We would be constructing a Many-to-one LSTM architecture, as such we require the data to be pre-processed into the necessary window size for training.

### Checkpoint 2: Display the dimensions for your pre-processed data and explain how the window size is incorporated into this structure.

In [ ]:
# Prepare dataset and labels for our training process
# where we use sequences of 10 glucose values to generate the 11th glucose value. 
# Keras requires input to be in the shape [samples, time steps, features].
def prepare_Xy(dataset, seq_length):
    X_train = []
    y_train = []
    for i in range(0, len(dataset) - seq_length, 1):
        seq_in = dataset[i:i + seq_length]
        seq_out = dataset[i + seq_length]
        X_train.append(seq_in)
        y_train.append(seq_out)
    seq_size = len(X_train)
    X_train = np.reshape(X_train, (seq_size, seq_length, 1))
    y_train = np.array(y_train)
        
    return X_train, y_train

In [ ]:
train_dict = {day: prepare_Xy(train[day], FIXED_WINDOW_SIZE) for day in range(len(train))}
test_dict = {day+len(train): prepare_Xy(test[day], FIXED_WINDOW_SIZE) for day in range(len(test))}

In [ ]:
train_dict, test_dict

Keras requires input to be in the shape [samples, time steps, features]. We have the following data structure as input into the model: $X = [[[x_{1}], ..., [x_{10}]], [[x_{2}], ..., [x_{11}]], ..., [[x_{n-11}], ..., [x_{n-1}]]]$, with the corresponding $y = [y_{1}, y_{2}, ..., y_{n}]$. Each day is a different continuous dataset, so the model has to be trained iteratively for each day. This is because we cannot assume that the true label of the last sequence for day 0 is the first value from day 1.

In this question, the window size refers to the number of time steps. Generally, if we have a window size of 10 and GlucoseValue as the only feature, we have a single labeled sample in the following form: $X[i] = [[x_{i}], [x_{i+1}], ... ,[x_{i+9}], [x_{i+10}]]$, and $y[i] = x_{i+11}$.

Extra note to self: If there are multiple features such as HeartRate, we could keep the same window size and easily add modify the data structure as such: $X[i] = [[x_{i}, u_{i}], [x_{i+1}, u_{i+1}], ... ,[x_{i+9}, u_{i+9}], [x_{i+10}, u_{i+10}]]$, and $y[i] = x_{i+11}$.

### Checkpoint 3: Explain how this many-to-one structure presented is incorporated in your data structure.

Following the above description, we require 10 timesteps to make a single prediction during training phase. We only care about the final output state at the final timestamp. Every other hidden output state in between is ignored.

<img src="many-to-one-archi.JPG" width="50%" align="center"/>

# Modelling and Training

### Checkpoint 4: Select the correct Loss Function and optimiser and explain the reason for your choice.

I use mean squared error because my output is an unbounded continuous value. Using this error function penalizes greater difference between predicted output and the actual output more severely. This will help to reduce deviation. However, I also keep track of mean absolute error, which is a metric that is easier for users to comprehend.

I use Adam due to its ability to leverage the power of adaptive learning rates methods to find individual learning rates for each parameter. Specifically, it uses the squared gradients to scale the learning rate like RMSprop and it takes advantage of momentum by using moving average of the gradient instead of gradient itself like SGD with momentum. As the algorithm combines the advantages of several other optimizers, choosing Adam therefore sounds like a reasonable approach to begin with.

Note on validation data:
1. Instead of creating a separate validation set based on a full day dataset, I split each train data into train and validation during training.
2. This allows me to keep track of the train and validation performance by each day, as the model is being trained iteratively.

In [ ]:
# Construct our LSTM model

def build_model(window_size):
    model = Sequential()
    model.add(LSTM(256, input_shape=(window_size, 1)))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    return model

def train_model(model, train_dict, epochs=30, batch_size=4, callbacks=[]):
    train_losses = []
    val_losses = []

    for k, (X,y) in train_dict.items():
        print("=============== TRAINING ON DAY {} ===============".format(k))
        model_history = model.fit(X, y, validation_split=0.2, epochs=epochs, batch_size=batch_size, verbose=1, callbacks=callbacks)
        train_losses.append(model_history.history['loss'])
        val_losses.append(model_history.history['val_loss'])
    
    return flatten(train_losses), flatten(val_losses)

In [ ]:
# Define callbacks

es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=10)
mc = ModelCheckpoint('best_weights.h5', monitor='val_loss', save_best_only=True, save_weights_only=True)

In [ ]:
# Train our LSTM model

model = build_model(FIXED_WINDOW_SIZE)
train_losses, val_losses = train_model(model, train_dict, callbacks=[es, mc])

In [ ]:
# Load best weights
model.load_weights('best_weights.h5')

In [ ]:
# sanity check
test_input = np.array([[[53],[57],[48],[51],[52],[45],[47],[52],[54],[55]]])
model.predict(test_input)

### Checkpoint 5: Graph and display the training loss of the model.

In [ ]:
def performance_graph(train_loss_hist, val_loss_hist):
    plt.plot(train_loss_hist, 'g', label='Training')
    plt.plot(val_loss_hist, 'b', label='Validation')
    plt.title('Training and Validation loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:
performance_graph(train_losses, val_losses)

# Validation
Now that we have our trained model, we would want to validate it on the rest of the remaining data to ensure that our model is not only accurate on a single session data but for mutiple other instance. Utilise the model on the remaining data available and show that the model accuracy is acceptable.

### Checkpoint 6: Graph the remaining instances and plot them
Provide the true data and the validated data on the same graph, display the mean loss for each of the instance.

In [ ]:
def evaluation_graph(X, y):
    _, test_mae = model.evaluate(X,y)
    pred_y = model.predict(X)
    
    plt.plot(y, 'g', label='True')
    plt.plot(pred_y, 'b', label='Predicted')
    plt.title('Mean Absolute Loss = {}'.format(test_mae))
    plt.xlabel('Timestep')
    plt.ylabel('Glucose Value')
    plt.legend()
    plt.show()

In [ ]:
for k, (X,y) in test_dict.items():
    print("=============== TESTING ON DAY {} ===============".format(k))
    evaluation_graph(X, y)

# Optimisation
We would now want to optimise the prediction accuracy by carrying out hyperparameter optimisation. Vary the windows size of the model and find the optimal window size that gives back the lowest loss.

### Checkpoint 7: Write a short report presenting your analysis on the optimal hyper-parameter of choice. You may include the necessary graphs or printout to explain your optimal hyper-parameter of choice.

Hyperparameter tuning should only be based on validation data, and not the train or test data. The most optimal hyperparameter is the one that produces the lowest validation loss.

In [ ]:
window_sizes = [5, 10, 15, 20, 25, 30]

best_size = None
lowest_loss = float('inf')
window_perf_dict = dict()

for size in window_sizes:
    print("=============== WINDOW SIZE {} ===============".format(size))
    model = build_model(size)
    train_dict = {day: prepare_Xy(train[day], size) for day in range(len(train))}
    _, val_losses = train_model(model, train_dict)
    model_loss = min(val_losses) # since we save the model weights by their lowest validation loss
    window_perf_dict[size] = model_loss
    
    if model_loss < lowest_loss:
        lowest_loss = model_loss
        best_size = size

In [ ]:
def hyperparmeter_graph(data):
    plt.bar(range(len(data)), list(data.values()), align='center')
    plt.xticks(range(len(data)), list(data.keys()))
    plt.title("Validation loss vs window sizes")
    plt.xlabel('Window sizes')
    plt.ylabel('Validation loss')
    plt.show()

In [ ]:
print("The most optimal window size is {}".format(best_size))
hyperparmeter_graph(window_perf_dict)

Add analysis